In [1]:
"""Modular aircraft concept"""
import pickle
import numpy as np
from gpkit import Model, Variable, Vectorize
from gpkit.constraints.tight import Tight
from gpkit.constraints.loose import Loose
from gpkit.constraints.bounded import Bounded

from gpkit import ureg as u

from aircraft import *

from prettytable import PrettyTable

In [2]:
class SimpleAircraft(Model):
    
    def setup(self):
        
        constraints = []
        components  = []
        
        W_0       = self.W_0       = Variable('W_0', "N", "Weight (force) of aircraft")
        W_payload = self.W_payload = Variable('W_{payload}', "N", "Weight (force) of payload + crew")
        W_empty   = self.W_empty   = Variable('W_{empty}', "N", "Weight (force) of empty aircraft")
        #W_fuel    = self.W_fuel    = Variable('W_{fuel}', "N", "Weight (force) of fuel")
        W_dry     = self.W_dry     = Variable('W_{dry}', "N", "Weight (force) of dry aircraft (payload + empty)")
        
        M_0       = self.M_0       = Variable('M_0', "kg", "Mass of aircraft")
        M_payload = self.M_payload = Variable('M_{payload}', "kg", "Mass of payload + crew")
        M_empty   = self.M_empty   = Variable('M_{empty}', "kg", "Mass of empty aircraft")
        #M_fuel    = self.M_fuel    = Variable("M_{fuel}", "kg", "Mass of fuel")
        M_dry     = self.M_dry     = Variable("M_{dry}", "kg", "Mass of dry aircraft (payload + empty)")
        
        g_0 = self.g_0 = Variable("g_0", 9.81, "m/s^2", "Acceleration due to gravity")

        #constraints += [Tight([W_0 >= W_dry + W_fuel])]
        constraints += [Tight([W_dry >= W_payload + W_empty])]

        constraints += [Tight([W_0 == M_0*g_0])]
        constraints += [Tight([W_dry == M_dry*g_0])]
        #constraints += [Tight([W_fuel == M_fuel*g_0])]
        constraints += [Tight([W_payload == M_payload*g_0])]
        constraints += [Tight([W_empty == M_empty*g_0])]
        
        #payload mass estimate
        N_crew = self.N_crew = Variable("N_{crew}", 4., "", "number of crew")
        N_passengers = self.N_passengers = Variable("N_{passengers}", 50., "", "number of passengers")
        M_per_person = Variable("M_{per passenger}", 100., "kg", "mass per passenger") # todo: get real number, currently arbitrary
        
        constraints += [M_payload >= M_per_person*(N_crew + N_passengers)] # todo: refine this mass estimate
        #constraints += [M_payload >= 9500*u.kg] # todo: refine this mass estimate
        #todo this 9500 comes from CRJ500 -> using 54*100 is far smaller!!

        
        # empty weight fraction
        fit_A =  0.97 #Using Table 3.1 of Raymer, Jet transport
        fit_C = -0.06 #Using Table 3.1 of Raymer, Jet transport
        fit_K_vs = 1.00 # assumes fixed sweep
        constraints += [Tight([M_empty/M_0 >= fit_A*(M_0/(1.*u.kg))**fit_C*fit_K_vs])]
                    
        M_1 = Variable("M_1", "kg", "Mass at 1")
        M_2 = Variable("M_2", "kg", "Mass at 2")
        M_3 = Variable("M_3", "kg", "Mass at 3")
        M_4 = Variable("M_4", "kg", "Mass at 4")
        M_5 = Variable("M_5", "kg", "Mass at 5")
        M_6 = Variable("M_6", "kg", "Mass at 6")
        M_7 = Variable("M_7", "kg", "Mass at 7")
        M_8 = Variable("M_8", "kg", "Mass at 8")
        M_9 = Variable("M_9", "kg", "Mass at 9")
        #M_dry for the last position (without the ullage)
        
        # standard fuel fraction constraints:
        #warmup and takeoff: 0.970
        #climb: 0.985
        #landing: 0.995
        constraints += [M_1/M_0 <= 0.97]
        constraints += [M_2/M_1 <= 0.985]
        #constraints += [M_3/M_2 >= ]     #cruise to target
        constraints += [M_4/M_3 <= 0.995]
        constraints += [M_5/M_4 <= 0.985]
        #constraints += [M_6/M_7 >= ]     #cruise to alt
        #constraints += [M_7/M_6 >= 0.99] #loiter
        constraints += [M_8/M_7 <= 0.995] # Raymer eq.6.22
        constraints += [M_9/M_8 <= 0.995]
        constraints += [M_9 >= M_dry*1.06] #tank ullage constraint
        
        #constraints += [M_0 >= M_dry + M_fuel]

        

        
        # main cruise range equation
        R_cruise = self.R_cruise = Variable("R_{cruise}", 2000., "km", "Main range of aircraft")
        V_cruise = self.V_cruise = Variable("V_{cruise}", "m/s", "Cruise speed of aircraft")
        Ma_cruise = self.Ma_cruise = Variable("Ma_{cruise}", 0.75, "", "Cruise mach number")
        h_cruise  = self.h_cruise = Variable("h_{cruise}", 35000., "ft", "Cruise altitude")
        a_cruise = Variable("a_{cruise}", 295.2, "m/s", "Speed of sound at 36,089 ft") #https://www.engineeringtoolbox.com/elevation-speed-sound-air-d_1534.html
        
        LD_max = Variable("LD_{max}", "", "L/D max") #todo: eyeballed from Raymer Fig 3.5. Need to incorporate better approx
        
        S_wet_S_ref = Variable("S_{wet}/S_{ref}", 6., "", "S wet to S ref ratio") # eyeballed from Raymer Fig. 3.6
        AR = Variable("AR", 8., "", "Aspect ratio, main") #todo: arbitrary
        
        K_LD = Variable("K_LD", 15.5, "", "Coefficient for estimating max lift to drag")
        constraints += [Tight([LD_max <= K_LD*(AR/(S_wet_S_ref))])]
        
        SFC_cruise = self.SFC_cruise = Variable("SFC_{cruise}", 19.8, "mg/N/s", "Specific Fuel Consumption, Cruise") #Table 3.3 of Raymer, for Low bypass Turbofan
        
        z1 = Variable("z1", "", "Dummy variable for range eqn1")
        constraints += [z1 >= R_cruise*SFC_cruise*g_0/(V_cruise*(0.866*LD_max))]
        constraints += [M_2/M_3 >= 1 + z1 + z1**2/2 + z1**3/6 + z1**4/24]
        
        
        # alt cruise range equation
        R_alt = self.R_alt = Variable("R_{alt}", 370., "km", "Alt range of aircraft")
        V_alt = self.V_alt = Variable("V_{alt}", "m/s", "Alt speed of aircraft for alternative")
        Ma_alt = self.Ma_alt = Variable("Ma_{alt}","", "Alt mach number")
        h_alt  = self.h_alt = Variable("h_{alt}", "ft", "Alt cruise altitude")
        a_alt = Variable("a_{alt}",295.2, "m/s", "Speed of sound at alt ft") #https://www.engineeringtoolbox.com/elevation-speed-sound-air-d_1534.html
        
        z2 = Variable("z2", "", "Dummy variable for range eqn 2")
        constraints += [z2 >= R_alt*SFC_cruise*g_0/(V_alt*(0.866*LD_max))]
        constraints += [M_5/M_6 >= 1 + z2 + z2**2/2 + z2**3/6 + z2**4/24]

        
        constraints += [Tight([V_cruise <= Ma_cruise*a_cruise])]
        constraints += [Tight([V_alt <= Ma_alt*a_alt])]
        constraints += [Ma_alt <= 0.8]

        
        #loiter
        t_loiter = self.t_loiter = Variable("t_{loiter}", 45, "min", "loiter endurance")
        
        LD_max = Variable("LD_{max}", "", "L/D max") #todo: eyeballed from Raymer Fig 3.5. Need to incorporate better approx
        
        S_wet_S_ref = Variable("S_{wet}/S_{ref}", 6., "", "S wet to S ref ratio") # eyeballed from Raymer Fig. 3.6
        AR = Variable("AR", 8., "", "Aspect ratio, main") #todo: arbitrary
        
        K_LD = Variable("K_LD", 15.5, "", "Coefficient for estimating max lift to drag")
        constraints += [Tight([LD_max <= K_LD*(AR/(S_wet_S_ref))])]
        
        SFC_cruise = self.SFC_cruise = Variable("SFC_{cruise}", 19.8, "mg/N/s", "Specific Fuel Consumption, Cruise") #Table 3.3 of Raymer, for Low bypass Turbofan
        
        z3 = Variable("z3", "", "Dummy variable for loitter eqn")
        constraints += [z3 >= t_loiter*SFC_cruise*g_0/(LD_max)]
        constraints += [M_6/M_7 >= 1 + z3 + z3**2/2 + z3**3/6 + z3**4/24]
        
        
        
        return constraints, components
            
        
        

In [3]:
#create the simple aircraft
AC = SimpleAircraft()

# define the optimizer to the AC.W_0, and set constraints to be the AC
M = Model(AC.W_0, Bounded(AC))

#print latex version of the constraints
M

<gpkit.Model object containing 1 top-level constraint(s) and 40 variable(s)>

In [4]:
# run a solve
sol = M.solve()

Using solver 'cvxopt'
Solving for 24 variables.
Solving took 0.102 seconds.


In [34]:
(AC.SFC_cruise)

gpkit.Variable(SimpleAircraft.SFC_{cruise} [mg/N/s])

In [40]:
(19.8*u.mg/(u.N*u.s) * 9.81*u.m/u.s**2) * 1*u.kg/1000000*u.mg

0.00019423800000000003 <Unit('kilogram * meter * milligram ** 2 / newton / second ** 3')>

In [5]:
# print(sol.summary()) 
#the summary which usually works well is a bit buggy here. Ive raised an appropriate github issue, and hopefully they will push a new version of gpkit that fixes it. 
#In the mean time ive written a rough version of it

In [24]:
m_fuel = sol['freevariables']['SimpleAircraft.M_0']-sol['freevariables']['SimpleAircraft.M_9']
m_fuel

4568.814541781234

In [25]:
fuel_fraction = m_fuel/sol['freevariables']['SimpleAircraft.M_0']
fuel_fraction

0.1943797116501431

In [6]:
x = PrettyTable()

x.field_names = ['Type', 'Var', 'Val', 'Unit', "Description"]

for k, v in sol['freevariables'].items():
    
    x.add_row(['Free', k, v, k.unitstr(), k.descr['label']])

    
for k, v in sol['constants'].items():
    
    x.add_row(['Fix', k, v, k.unitstr(), k.descr['label']])

x.float_format = '10.2'

print(x)

+------+----------------------------------+------------+--------+--------------------------------------------------+
| Type |               Var                |    Val     |  Unit  |                   Description                    |
+------+----------------------------------+------------+--------+--------------------------------------------------+
| Free |        SimpleAircraft.W_0        |  230579.98 |   N    |            Weight (force) of aircraft            |
| Free |    SimpleAircraft.W_{payload}    |   52974.00 |   N    |         Weight (force) of payload + crew         |
| Free |      SimpleAircraft.W_{dry}      |  175245.20 |   N    | Weight (force) of dry aircraft (payload + empty) |
| Free |     SimpleAircraft.W_{empty}     |  122271.20 |   N    |         Weight (force) of empty aircraft         |
| Free |        SimpleAircraft.M_0        |   23504.59 |   kg   |                 Mass of aircraft                 |
| Free |      SimpleAircraft.M_{dry}      |   17863.93 |   kg   

In [7]:
vars = ['SimpleAircraft.M_0','SimpleAircraft.M_1', 'SimpleAircraft.M_2','SimpleAircraft.M_3','SimpleAircraft.M_4','SimpleAircraft.M_5','SimpleAircraft.M_6','SimpleAircraft.M_7','SimpleAircraft.M_8','SimpleAircraft.M_9', 'SimpleAircraft.M_{dry}']
alpha_list = [sol['freevariables'][var]/sol['freevariables']['SimpleAircraft.M_0'] for var in vars]

In [8]:
alpha_list

[1.0,
 0.97000000009258,
 0.9554500001809966,
 0.8662240952899448,
 0.8618929748936808,
 0.8489645803500689,
 0.8346511575462172,
 0.8137373179464782,
 0.8096686314320698,
 0.8056202883498569,
 0.7600191400272281]

In [9]:
#full solution dictionary
sol

{'freevariables': {SimpleAircraft.W_0: 230579.98318284622,
  SimpleAircraft.W_{payload}: 52973.99996394035,
  SimpleAircraft.W_{dry}: 175245.20054952195,
  SimpleAircraft.W_{empty}: 122271.20200497401,
  SimpleAircraft.M_0: 23504.58544770596,
  SimpleAircraft.M_{dry}: 17863.934818661983,
  SimpleAircraft.M_{payload}: 5399.999997580617,
  SimpleAircraft.M_{empty}: 12463.93496867222,
  SimpleAircraft.M_1: 22799.447886450835,
  SimpleAircraft.M_2: 22457.456170264908,
  SimpleAircraft.M_4: 20258.437075166006,
  SimpleAircraft.M_3: 20360.238264604297,
  SimpleAircraft.M_5: 19954.560520914027,
  SimpleAircraft.M_8: 19030.925531822228,
  SimpleAircraft.M_7: 19126.558321660068,
  SimpleAircraft.M_9: 18935.770905924725,
  SimpleAircraft.LD_{max}: 20.666666586412028,
  SimpleAircraft.V_{cruise}: 221.39999959520512,
  SimpleAircraft.z1: 0.0980388600349257,
  SimpleAircraft.V_{alt}: 236.15994026077456,
  SimpleAircraft.z2: 0.017003621183977264,
  SimpleAircraft.M_6: 19618.12945157175,
  SimpleAirc

In [10]:
# this is for the more complicated airplane, so ignore it.

#AC = Aircraft()
#MISSION = Mission(AC)
#M = Model(MISSION.takeoff_fuel, [MISSION, AC])

In [11]:
#AC

In [12]:
#print(M)

In [13]:
#sol = M.solve(verbosity=0)
#print(sol.summary())

In [14]:
############ tool to check fit against historical data

In [15]:
fit_A =  0.97 #Using Table 3.1 of Raymer, Jet transport
fit_C = -0.06 #Using Table 3.1 of Raymer, Jet transport
fit_K_vs = 1.00 # assumes fixed sweep

In [16]:
#ERJ145
me = 12114
m0 = 22000
actual_ratio = me/m0
expected_ratio = fit_A*(m0)**fit_C*fit_K_vs
print(actual_ratio, expected_ratio)
maxTW = 0.31
maxTW

0.5506363636363636 0.5323856898319124


0.31

In [17]:
#crj200
me = 13835
m0 = 24041
actual_ratio = me/m0
expected_ratio = fit_A*(m0)**fit_C*fit_K_vs
print(actual_ratio, expected_ratio)
maxTW = 38840/(m0*9.81)
maxTW

0.5754752298157314 0.5295592794391422


0.16468638077975947

In [18]:
#crj700
me = 20069
m0 = 34019
actual_ratio = me/m0
expected_ratio = fit_A*(m0)**fit_C*fit_K_vs
print(actual_ratio, expected_ratio)
maxTW = 61300/(m0*9.81)
maxTW


0.5899350363032423 0.5186429056203041


0.18368340603810204